In [1]:
!pip install hmmlearn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter, defaultdict
from hmmlearn import hmm
import pickle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 4.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ML_Hackathon')

# Verify files exist
!ls -lh



Mounted at /content/drive
total 796K
-rw------- 1 root root  13K Nov  3 16:27 1_Data_Preparation_and_HMM.ipynb
-rw------- 1 root root 178K Nov  3 11:02 2_RL_Agent_Training.ipynb
-rw------- 1 root root  12K Nov  3 11:06 3_Evaluation_and_Analysis.ipynb
-rw------- 1 root root 562K Nov  3 03:18 corpus.txt
drwx------ 2 root root 4.0K Nov  3 03:37 models
drwx------ 2 root root 4.0K Nov  3 03:48 results
-rw------- 1 root root  23K Nov  3 03:18 test.txt


In [3]:
with open('corpus.txt', 'r') as f:
    corpus_words = [line.strip().lower() for line in f.readlines() if line.strip()]

with open('test.txt', 'r') as f:
    test_words = [line.strip().lower() for line in f.readlines() if line.strip()]

print(f"Corpus words: {len(corpus_words)}")
print(f"Test words: {len(test_words)}")
print(f"\nSample corpus words: {corpus_words[:10]}")
print(f"\nWord length distribution:")
# Analyze word lengths
from collections import Counter
length_dist = Counter([len(w) for w in corpus_words])
for length in sorted(length_dist.keys())[:10]:
    print(f"  {length} letters: {length_dist[length]} words")



Corpus words: 50000
Test words: 2000

Sample corpus words: ['suburbanize', 'asmack', 'hypotypic', 'promoderationist', 'consonantly', 'philatelically', 'cacomelia', 'thicklips', 'luciferase', 'cinematography']

Word length distribution:
  1 letters: 46 words
  2 letters: 84 words
  3 letters: 388 words
  4 letters: 1169 words
  5 letters: 2340 words
  6 letters: 3755 words
  7 letters: 5111 words
  8 letters: 6348 words
  9 letters: 6808 words
  10 letters: 6465 words


In [4]:
import random

class HangmanGame:
    def __init__(self, word_list, max_wrong_guesses=6):
        self.word_list = word_list
        self.max_wrong_guesses = max_wrong_guesses
        self.reset()

    def reset(self, word=None):
        self.secret_word = word if word else random.choice(self.word_list)
        self.guessed_letters = set()
        self.wrong_guesses = 0
        self.repeated_guesses = 0
        self.game_over = False
        self.won = False
        return self.get_state()

    def get_state(self):
      masked_word = ''.join([c if c in self.guessed_letters else '_'
                            for c in self.secret_word])
      return {
          'masked_word': masked_word,
          'guessed_letters': list(self.guessed_letters),
          'wrong_guesses': self.wrong_guesses,
          'lives_remaining': self.max_wrong_guesses - self.wrong_guesses
      }

    def make_guess(self, letter):
        letter = letter.lower()

        # Check if already guessed
        if letter in self.guessed_letters:
            self.repeated_guesses += 1
            return -2, self.game_over

        self.guessed_letters.add(letter)

        # Check if letter in word
        if letter in self.secret_word:
            # Check if won
            if all(c in self.guessed_letters for c in self.secret_word):
                self.won = True
                self.game_over = True
                return 20, True
            return 2, False
        else:
            self.wrong_guesses += 1
            if self.wrong_guesses >= self.max_wrong_guesses:
                self.game_over = True
                return -10, True
            return -1, False


In [5]:
class SmartAgent:
    """Probabilistic agent using letter patterns from corpus"""

    def __init__(self, corpus_words):
        self.corpus_words = corpus_words
        self.build_models()

    def build_models(self):
        """Build all probabilistic models"""
        # Letter frequency
        self.letter_freq = Counter(''.join(self.corpus_words))

        # Bigram transitions
        self.bigrams = defaultdict(Counter)
        for word in self.corpus_words:
            for i in range(len(word)-1):
                self.bigrams[word[i]][word[i+1]] += 1

        # Position models
        self.start_letters = Counter([w[0] for w in self.corpus_words if w])
        self.end_letters = Counter([w[-1] for w in self.corpus_words if w])

    def get_letter_probabilities(self, masked_word, guessed_letters):
        """Return probability distribution over unguessed letters"""
        from collections import Counter

        scores = Counter()
        revealed = len([c for c in masked_word if c != '_'])
        total = len(masked_word)
        progress = revealed / total if total > 0 else 0

        # Phase 1: Early game (0-40%) - prioritize vowels
        if progress < 0.4:
            for v in 'eaoiu':
                if v not in guessed_letters:
                    scores[v] = 10

        # Phase 2: Mid game (40-50%) - common consonants
        if progress < 0.5:
            for c in 'tnsr':
                if c not in guessed_letters:
                    scores[c] = 5

        # Phase 3: Bigram patterns
        for i, char in enumerate(masked_word):
            if char == '_':
                if i > 0 and masked_word[i-1] != '_':
                    prev = masked_word[i-1]
                    if prev in self.bigrams:
                        for next_letter, count in self.bigrams[prev].most_common(10):
                            if next_letter not in guessed_letters:
                                scores[next_letter] += count / 100

        # Phase 4: Overall letter frequency
        for letter, count in self.letter_freq.items():
            if letter not in guessed_letters:
                scores[letter] += count / 10000

        # Convert to probabilities
        total_score = sum(scores.values())
        if total_score == 0:
            for i, letter in enumerate('etaoinshrdlcumwfgypbvkjxqz'):
                if letter not in guessed_letters:
                    scores[letter] = 26 - i
            total_score = sum(scores.values())

        return {letter: score/total_score for letter, score in scores.items()}

    def get_guess(self, masked_word, guessed_letters):
        """Get best letter guess"""
        probs = self.get_letter_probabilities(masked_word, guessed_letters)
        if probs:
            return max(probs.items(), key=lambda x: x[1])[0]
        return 'e'

print("✅ SmartAgent class defined")


✅ SmartAgent class defined


In [6]:
def evaluate_strategy(agent, word_list, num_games=100, verbose=True):
    """Evaluate an agent on word list"""
    wins = 0
    total_wrong = 0
    total_repeated = 0

    for i, word in enumerate(word_list[:num_games]):
        game = HangmanGame([word])
        state = game.reset(word=word)

        steps = 0
        while not game.game_over and steps < 30:
            guess = agent.get_guess(state['masked_word'], set(state['guessed_letters']))
            reward, done = game.make_guess(guess)
            state = game.get_state()
            steps += 1

        if game.won:
            wins += 1
        total_wrong += game.wrong_guesses
        total_repeated += game.repeated_guesses

    success_rate = wins / num_games

    if verbose:
        print(f"\n{'='*50}")
        print(f"Evaluation Results ({num_games} games)")
        print(f"{'='*50}")
        print(f"Wins: {wins}/{num_games} ({success_rate*100:.1f}%)")
        print(f"Total Wrong Guesses: {total_wrong}")
        print(f"Total Repeated: {total_repeated}")
        print(f"{'='*50}")

    return {
        'wins': wins,
        'success_rate': success_rate,
        'total_wrong': total_wrong,
        'total_repeated': total_repeated
    }

print("✅ Evaluation function defined")


✅ Evaluation function defined


In [7]:
print("\n" + "="*60)
print("BUILDING SMARTAGENT")
print("="*60)

smart_agent = SmartAgent(corpus_words)
print("✅ SmartAgent built and trained on corpus")

# Test on sample
print("\nTesting on 100 corpus words...")
corpus_test = evaluate_strategy(smart_agent, corpus_words[-100:], num_games=100, verbose=True)

print("\nTesting on 100 test words...")
test_results = evaluate_strategy(smart_agent, test_words, num_games=100, verbose=True)

print(f"\n📊 Summary:")
print(f"   Corpus performance: {corpus_test['success_rate']*100:.1f}%")
print(f"   Test performance: {test_results['success_rate']*100:.1f}%")



BUILDING SMARTAGENT
✅ SmartAgent built and trained on corpus

Testing on 100 corpus words...

Evaluation Results (100 games)
Wins: 26/100 (26.0%)
Total Wrong Guesses: 546
Total Repeated: 0

Testing on 100 test words...

Evaluation Results (100 games)
Wins: 28/100 (28.0%)
Total Wrong Guesses: 540
Total Repeated: 0

📊 Summary:
   Corpus performance: 26.0%
   Test performance: 28.0%


In [8]:
print("\n" + "="*60)
print("CORPUS vs TEST SET ANALYSIS")
print("="*60)

corpus_set = set(corpus_words)
test_set = set(test_words)
overlap = corpus_set.intersection(test_set)

print(f"\n1. VOCABULARY OVERLAP:")
print(f"   Corpus words: {len(corpus_set):,}")
print(f"   Test words: {len(test_set):,}")
print(f"   Common words: {len(overlap):,}")
print(f"   Overlap %: {len(overlap)/len(test_set)*100:.2f}%")

print(f"\n2. UNIQUE TEST WORDS:")
unique_test = test_set - corpus_set
print(f"   Completely unseen: {len(unique_test):,}")
print(f"   Examples: {list(unique_test)[:5]}")

print(f"\n3. IMPLICATIONS:")
print(f"   ✓ SmartAgent CANNOT memorize test words")
print(f"   ✓ Must rely on learned patterns (bigrams, frequency)")
print(f"   ✓ This explains 97% → 26% drop (corpus → test)")
print("="*60)



CORPUS vs TEST SET ANALYSIS

1. VOCABULARY OVERLAP:
   Corpus words: 49,398
   Test words: 2,000
   Common words: 0
   Overlap %: 0.00%

2. UNIQUE TEST WORDS:
   Completely unseen: 2,000
   Examples: ['niccoliferous', 'thereamong', 'releasable', 'resident', 'torchless']

3. IMPLICATIONS:
   ✓ SmartAgent CANNOT memorize test words
   ✓ Must rely on learned patterns (bigrams, frequency)
   ✓ This explains 97% → 26% drop (corpus → test)


In [9]:
# Save model
with open('models/smart_agent.pkl', 'wb') as f:
    pickle.dump(smart_agent, f)

print("✅ SmartAgent saved to models/smart_agent.pkl")
print(f"✅ Ready for RL training in Notebook 2")


✅ SmartAgent saved to models/smart_agent.pkl
✅ Ready for RL training in Notebook 2
